You can install keras independently or you can use the one that comes with tensorflow

In [ ]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [ ]:
data = pd.read_csv('./../data/wine-quality/winequalityN.csv')
data.head()

Identify number of unique categories in target variable: quality

In [ ]:
data.type.unique()

In [ ]:
data.shape

In [ ]:
data.describe()

#### Checking for any NaN entries

In [ ]:
data.isnull().sum()

In [ ]:
data = data.fillna(0.0)
data.isnull().sum()

#### Seperating features and target

In [ ]:
data_cols = data.columns
features = data[data_cols[data_cols != 'type']]
n_cols = features.shape[1]
target = data['type']

#### Normalizing features

In [ ]:
features = (features - features.mean()) / features.std()
features.head()

In [ ]:
target.head()

In [ ]:
# We cannot use string white and red as categories directly
# We will encode them
# white: 1
# red: 0
target = target.replace('white',1)
target = target.replace('red', 0)
target

### Dividing the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                            test_size=0.20, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [ ]:
target_train = to_categorical(target_train)

In [ ]:
target_test = to_categorical(target_test)

In [ ]:
num_class = target_train.shape[1]
num_class

### Building a Sequential Model

In [ ]:
def classification():
    model = Sequential()
    #Adding layers to the model
    # 2 hidden layers of 5 units eacha
    model.add(Dense(5, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(num_class, activation='softmax'))
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

### Training the model

In [ ]:
model = classification()
history = model.fit(features_train, target_train, epochs=50, validation_split=0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist

### Plotting training versur Validation Loss
The two loss curves and the final loss values should be almost identical.

In [ ]:
plt.figure(figsize=(12,6))

ax0 = plt.subplot(1,2,1)
ax0.set_title('Training Loss Vs Validataion Loss')
ax0 = sns.lineplot(history.epoch, hist.loss, label='Training Loss')
ax0 = sns.lineplot(history.epoch, hist.val_loss, label='Validation Loss')
ax0.set_xlabel('Epochs')
ax0.set_ylabel('Loss')

ax1 = plt.subplot(1,2,2)
ax1.set_title('Training Accuracy Vs Validataion Accuracy')
ax1 = sns.lineplot(history.epoch, hist.accuracy, label='Training Accuracy')
ax1 = sns.lineplot(history.epoch, hist.val_accuracy, label='Validation Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
plt.show()

### Testing the model

In [ ]:
#model.evaluate returns test loss or list of scalars (if the model has multiple outputs and/or metrics)
evaluation = model.evaluate(features_test, target_test)
print("Model's loss on test data : {} \nModel's accuracy on test data : {}".format(evaluation[0], evaluation[1]))

### Saving the learned Model

In [ ]:
import os
if not os.path.isdir('./models/keras-classificaton'):
    os.makedirs('./models/keras-classificaton')
model.save('./models/keras-classificaton/classification.h5')

### Loading the pre-trained model

In [ ]:
from keras.models import load_model
pretrained_model = load_model('./models/keras-classificaton/classification.h5')
pretrained_model

In [ ]:
pretrained_model.evaluate(features_test, target_test)

### Prediction from learned model
Softmax outputs the probability of each class, and sum of probability of all the class equals to 1 for each instance

In [ ]:
predictions = model.predict(features_test)
predictions